In [1]:
import sys
sys.path.append('..')

In [2]:
import pandas as pd
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, DateType, TimestampType
from clean.load import Load
from clean.transform import Transform

import pyspark
import pyspark.pandas as ps
from pyspark.sql import SparkSession

/home/vinicius/teste/raizen_test/etl_pipeline_test_2023/venv/lib/python3.10/site-packages/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
load = Load()

In [4]:
dfs = load.load_all()

In [5]:
t = Transform()

In [6]:
for df in dfs:
    t.add_raw_df(df)

In [7]:
dfs = t.get_final_table()

In [8]:
df = pd.concat(dfs, ignore_index=True)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
df

In [ ]:
spark = SparkSession.builder.appName('PySpark').getOrCreate()

In [ ]:
sp_df = spark.createDataFrame(df)

In [ ]:
sp_df.show()

In [ ]:
sp_df

## Schema

Data should be stored in the following format:

| Column       | Type        |
| ------------ | ----------- |
| `year_month` | `date`      |
| `uf`         | `string`    |
| `product`    | `string`    |
| `unit`       | `string`    |
| `volume`     | `double`    |
| `created_at` | `timestamp` |

In [ ]:
# | `year_month` | `date`      |
# | `uf`         | `string`    |
# | `product`    | `string`    |
# | `unit`       | `string`    |
# | `volume`     | `double`    |
# | `created_at` | `timestamp` |

schema = StructType(
    [StructField("year_month", DateType(), True),
     StructField("uf", StringType(), True),
     StructField("product", StringType(), True),
     StructField("unit", StringType(), True),
     StructField("volume", DoubleType(), True),
     StructField("created_at", TimestampType(), True),
     ])
sp_df = spark.createDataFrame(df, schema=schema)
#Create DataFrame by changing schema
# sparkDF2 = spark.createDataFrame(pandasDF,schema=mySchema)
# sparkDF2.printSchema()
# sparkDF2.show()

In [ ]:
sp_df.printSchema()

In [ ]:
sp_df.write.option("header",True) \
        .partitionBy("year_month") \
        .mode("overwrite") \
        .parquet("vendas_y.parquet")


In [ ]:
sp_df.show()